In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-24 04:45:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-06-24 04:45:16 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("census_data_2000s").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://pollution-sight-data.s3.us-west-1.amazonaws.com/census_data_2000s.csv"
spark.sparkContext.addFile(url)
census_2000s_df = spark.read.csv(SparkFiles.get("census_data_2000s.csv"), header=True, inferSchema=True)
census_2000s_df.show()

+-------------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|               NAME| STNAME|POPESTIMATE2000|POPESTIMATE2001|POPESTIMATE2002|POPESTIMATE2003|POPESTIMATE2004|POPESTIMATE2005|POPESTIMATE2006|POPESTIMATE2007|POPESTIMATE2008|POPESTIMATE2009|POPESTIMATE2010|
+-------------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|            Alabama|Alabama|        4452173|        4467634|        4480089|        4503491|        4530729|        4569805|        4628981|        4672840|        4718206|        4757938|        4785298|
|     Abbeville city|Alabama|           2985|           2941|           2909|           2882|           2857|           2820|           2807|           2784|           2742|   

In [ ]:
census_2000s_df.printSchema()

root
 |-- NAME: string (nullable = true)
 |-- STNAME: string (nullable = true)
 |-- POPESTIMATE2000: integer (nullable = true)
 |-- POPESTIMATE2001: integer (nullable = true)
 |-- POPESTIMATE2002: integer (nullable = true)
 |-- POPESTIMATE2003: integer (nullable = true)
 |-- POPESTIMATE2004: integer (nullable = true)
 |-- POPESTIMATE2005: integer (nullable = true)
 |-- POPESTIMATE2006: integer (nullable = true)
 |-- POPESTIMATE2007: integer (nullable = true)
 |-- POPESTIMATE2008: integer (nullable = true)
 |-- POPESTIMATE2009: integer (nullable = true)
 |-- POPESTIMATE2010: integer (nullable = true)



In [ ]:
# Connect to AWS RDS instance
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection_string>:5432/<database_name>"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write census_2000s_df to table in RDS
census_2000s_df.write.jdbc(url=jdbc_url, table='census_data_2000s', mode=mode, properties=config)